## Search Engine With Tools and Agents

In [1]:
## Arxive--Research Papers
## Tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [2]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxive = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxive)
arxiv.name

'arxiv'

In [4]:
tools = [wiki, arxiv]

In [5]:
## Custom tools [RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=250).split_documents(docs)
vectordb = FAISS.from_documents(documents, embeddings)
retriever = vectordb.as_retriever()
retriever

c:\Users\Lenovo\anaconda3\envs\torchGPU\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021828484560>, search_kwargs={})

In [8]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "Langsmith-search", "Search any information about langsmith")
retriever_tool.name

'Langsmith-search'

In [9]:
tools = [wiki, arxiv, retriever_tool]

In [10]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Lenovo\\anaconda3\\envs\\torchGPU\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='Langsmith-search', description='Search any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002183D1A22A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000021

In [11]:
## Run all this tools with Agents and LLM Models
## Tools, LLM-->
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import os
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

In [12]:
llm = ChatGroq(groq_api_key=groq_api_key, model_name="gemma2-9b-it")

In [13]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [14]:
## Creating Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [15]:
## agent executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [18]:
agent_executor.invoke({"input": "Tell me about generative ai and why i need to learn it?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Generative AI'}`


Page: Generative artificial intelligence
Summary: Generative artificial intelligence (generative AI, GenAI, or GAI) is a subset of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data. These modeGenerative AI is a type of artificial intelligence that can create new content, like text, images, videos, and even code. 

There are many reasons why you might want to learn about generative AI:

* **Creativity:**  It can be used to generate creative content, such as writing stories, poems, or even music.
* **Efficiency:** It can automate tasks that would normally take a lot of time and effort, such as writing reports or creating marketing materials.
* **Innovation:** It is a rapidly evolving field with the potential to revolutionize many industries.
* **Understanding AI:** Learning about generative AI can give you a better understanding of h

{'input': 'Tell me about generative ai and why i need to learn it?',
 'output': "Generative AI is a type of artificial intelligence that can create new content, like text, images, videos, and even code. \n\nThere are many reasons why you might want to learn about generative AI:\n\n* **Creativity:**  It can be used to generate creative content, such as writing stories, poems, or even music.\n* **Efficiency:** It can automate tasks that would normally take a lot of time and effort, such as writing reports or creating marketing materials.\n* **Innovation:** It is a rapidly evolving field with the potential to revolutionize many industries.\n* **Understanding AI:** Learning about generative AI can give you a better understanding of how artificial intelligence works.\n\n\nWhether you're interested in using generative AI for creative purposes, professional development, or simply to stay ahead of the curve, learning about it can be a valuable investment. \n\n"}

In [19]:
agent_executor.invoke({"input": "Solve this equation step by step x^2+5x+6=0"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'quadratic formula'}`


Page: Quadratic formula
Summary: In elementary algebra, the quadratic formula is a closed-form expression describing the solutions of a quadratic equation. Other ways of solving quadratic equations, such as completing the square, yield the same solutThe quadratic formula is used to solve equations in the form ax² + bx + c = 0.  

The formula is:

x = (-b ± √(b² - 4ac)) / 2a

In our equation x² + 5x + 6 = 0, we have:

a = 1, b = 5, c = 6

Let's plug these values into the quadratic formula:

x = (-5 ± √(5² - 4 * 1 * 6)) / (2 * 1)

Simplify:

x = (-5 ± √(25 - 24)) / 2
x = (-5 ± √1) / 2
x = (-5 ± 1) / 2

This gives us two solutions:

x₁ = (-5 + 1) / 2 = -2

x₂ = (-5 - 1) / 2 = -3 



Therefore, the solutions to the equation x² + 5x + 6 = 0 are x = -2 and x = -3.


> Finished chain.


{'input': 'Solve this equation step by step x^2+5x+6=0',
 'output': "The quadratic formula is used to solve equations in the form ax² + bx + c = 0.  \n\nThe formula is:\n\nx = (-b ± √(b² - 4ac)) / 2a\n\nIn our equation x² + 5x + 6 = 0, we have:\n\na = 1, b = 5, c = 6\n\nLet's plug these values into the quadratic formula:\n\nx = (-5 ± √(5² - 4 * 1 * 6)) / (2 * 1)\n\nSimplify:\n\nx = (-5 ± √(25 - 24)) / 2\nx = (-5 ± √1) / 2\nx = (-5 ± 1) / 2\n\nThis gives us two solutions:\n\nx₁ = (-5 + 1) / 2 = -2\n\nx₂ = (-5 - 1) / 2 = -3 \n\n\n\nTherefore, the solutions to the equation x² + 5x + 6 = 0 are x = -2 and x = -3.\n"}

In [20]:
agent_executor.invoke({"input": "Solve this equation different way"})



> Entering new AgentExecutor chain...

Invoking: `Langsmith-search` with `{'query': 'solve equation different way'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

1. Install LangSmith​
PythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai
2. Create an API key​
To create an API key head to the Settings page. Then click Create API Key.
3. Set up your environment​
Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>
4. Log your first trace​
LangSmith + LangChain OSSYou don't need to use the LangSmith SDK directly if your application is built on LangChain/LangGraph (either Python and JS).See the how-to guide for tracing with LangChain here.
We provide multiple ways to log traces to LangSmith. Below, we'll highlight
how to use traceable(). See more on the Annotate code for tracing page.

PythonTypeScriptimport openaifrom langsm

{'input': 'Solve this equation different way',
 'output': 'I need more information to help you solve the equation. Please provide me with the equation you want to solve. \n\n\n'}

In [21]:
agent_executor.invoke({"input": "Tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `Langsmith-search` with `{'query': 'what is langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Click the link printed out by your evaluation run to access the LangSmith experiments UI,
and explore the results of your evaluation.
Learn more about evaluation in the tutorials, conceptual guide, and how-to guides.
Was this page helpful?You can leave detailed feedback on GitHub.NextConcepts1. Install LangSmith2. Create an API key3. Set up your environment4. Log your first trace5. Run your first evaluationCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.

1. Install LangSmith​
PythonTypeScriptpip install -U langsmith openaiyarn add langsmith openai
2. Create an API key​
To create an API key head to the Settings page. Then click Create API Key.
3. Set up your environment​
Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=

{'input': 'Tell me about langsmith',
 'output': "LangSmith is a platform for building production-grade LLM applications. \nIt allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nLangChain's open source frameworks langchain and langgraph work seemlessly with LangSmith but are not necessary - LangSmith works on its own! \n"}

In [22]:
agent_executor.invoke({"input": "Tell me about a research paper attentation all you need"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'attention all you need'}`


Published: 2024-07-22
Title: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models
Authors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini
Summary: The inference demand for LLMs has skyrThis sounds interesting! 

Could you please tell me more about what the paper says about how much attention is needed for inference?  


> Finished chain.


{'input': 'Tell me about a research paper attentation all you need',
 'output': 'This sounds interesting! \n\nCould you please tell me more about what the paper says about how much attention is needed for inference?  \n'}